# Краевая задача для обыкновенных дифференциальных уравнений второго порядка

# Постановка задачи (Вариант №5)

Определить первые четыре собственные значения задачи Штурма-Луивилля

$$ \begin{cases}
\dfrac{d^2y}{dx^2} + \lambda q_0(x)y = 0, \quad 0<x<D=1\\\\
y(0)=y(1) = 0\\
q_0(x) = \begin{cases} 2, & 0\leq x \leq 1/2 \\ 1, & 1/2 \leq x \leq 1\end{cases}
\end{cases}$$

предложив численный метод решения задачи. Реализовать его в виде программы, написанной на языке высокого уровня, произвести отладку на модельной задаче и расчёты с относительной точностью $\delta=10^{-4}$.

# Аналитическое решение модельной задачи

Рассмотрим вспомогательную модельную задачу с постоянным коэффициентом $q(x) \equiv 1$ на интервале $(0,1)$.

$$ \begin{cases}
\dfrac{d^2y}{dx^2} + \lambda y = 0, \quad 0<x<D=1\\\\
y(0)=y(1) = 0
\end{cases}$$

Общее решение данного дифференциального уравнения 

$$y(x) = C_1 \sin\left(\sqrt{\lambda} ~x\right) +  C_2 \cos\left(\sqrt{\lambda} ~x\right)$$

Подставляя его в начальные условия, получаем 

$$y(0) = C_2 = 0$$

$$y(1) = C_1 \sin(\sqrt{\lambda}) = 0$$

Второе уравнение имеет несколько решений:

1) $\sqrt{\lambda} = 0$

2) $C_1=0$, то есть $y_1(x)=0$ $-$ тривиальное решение.

3) $\sin\left(\sqrt{\lambda}\right) = 0$, следовательно $\sqrt{\lambda_n} = \pi n$, при $n=1,2,\ldots$.

Таким образом собственные числа и собственные равны

$$\lambda_n = \left(\pi n\right)^2, \quad n \in \mathbb{Z}_{++}$$

$$ y_n(x) = \sin\left(\sqrt{\lambda_n}x\right) $$

In [6]:
import numpy as np

X_0 = 0.5
LEFT_LIM  = 0.0
RIGHT_LIM = 1.0
LEFT_BOUND  = 0.0
RIGHT_BOUND = 1.0


def CheckX(x, func_name="CheckX"):
    if not isinstance(x, (int, long, float)):
        raise ValueError("Incorrect type of x parameter in function " + func_name)    
    if x < LEFT_LIM or x > RIGHT_LIM:
        raise ValueError("Parameter x = " + str(x) + " is out of (0,1) bounds in function " + func_name)    

def CheckN(n, func_name="CheckN"):
    if not isinstance(n, (int, long)):
        raise ValueError("Incorrect type of n parameter in function " + func_name)    
    if n < 0:
        raise ValueError("Parameter n = " + str(n) + " is <0 in function " + func_name)    
        
def QCoef(x):
    CheckX(x, 'QCoef')

    if x >= LEFT_LIM and x < X_0:
        return 2.0
    if x > X_0 and x <= RIGHT_LIM:
        return 1.0
    else:
        raise ValueError("x == X_0 in QCoef.")    
    
def QCoef_model(x):
    return 1.0

In [7]:
def AnalytEigVal_model(n):
    CheckN(n, 'AnalytEigVal_model')
    return (np.pi * n) ** 2

def AnalytEigFunc_model(n, x):
    CheckX(x, 'AnalytEigFunc_model')
    CheckN(n, 'AnalytEigFunc_model')
    return np.sin((AnalytEigVal_model(n) ** 0.5) * x) 

# Численное решение

Разобьём отрезок интегрирования $[0,1]$ равномерно на $L$ частей так, чтобы расстояние между узлами было постоянно и равно $h=1/L$. Получившаяся совокупность точек $x_l = lh$, $l = \overline{0,L}$. Сформулируем разностную задачу

$$ \begin{cases}
\dfrac{k_{l+1/2}(y_{l+1}-y_l) - k_{l-1/2}(y_l-y_{l-1})}{h} + \lambda q_0(x_l) y_l = 0, \quad l=\overline{1,L-1} \\
y_0 = 0 \\
y_L = 0
\end{cases}$$

In [8]:
class StatHeatCondProb:
    
    alpha = np.zeros(1)
    beta  = np.zeros(1)
    sol   = np.zeros(1)
    
    NODES_NUM = 0
    MESH_STEP = 0
    NODE_BEF_GAP = 0
    NODE_AFT_GAP = 0
    
    therm_cond_coef = 0
    q_coef = 0
    heat_source_coef = 0
    
    def __init__(self, nodes_num_, therm_cond_coef_, q_coef_, heat_source_coef_):
        self.NODES_NUM        = nodes_num_
        self.therm_cond_coef  =  therm_cond_coef_
        self.q_coef           = q_coef_
        self.heat_source_coef = heat_source_coef_
        
        self.MESH_STEP = (RIGHT_LIM - LEFT_LIM) / (self.NODES_NUM - 1)
        
        self.alpha = np.zeros(self.NODES_NUM)
        self.beta  = np.zeros(self.NODES_NUM)
        self.sol   = np.zeros(self.NODES_NUM)
        
        self.NODE_BEF_GAP = int(m.floor(X_0 / self.MESH_STEP)) # l_1
        self.NODE_AFT_GAP = int(m.ceil (X_0 / self.MESH_STEP))  # l_2
        
        return


    def CheckMeshPos(self, pos):
        if not isinstance(pos, (int, long)):
            raise ValueError("Incorrect type of pos parameter in function CheckMeshPos.") 
        if pos < 0 or pos > self.NODES_NUM:
            raise ValueError("Parameter pos is out of [0,NODES_NUM] bounds in function CheckMeshPos.") 
        return

    
    def coef_a(self, pos):
        self.CheckMeshPos(pos)
        return self.therm_cond_coef(self.MESH_STEP * (pos + 0.5))

    def coef_b(self, pos):
        self.CheckMeshPos(pos)
        return -(self.therm_cond_coef((pos + 0.5) * self.MESH_STEP) +\
                 self.therm_cond_coef((pos - 0.5) * self.MESH_STEP) +\
                 self.q_coef(pos * self.MESH_STEP) * (self.MESH_STEP ** 2))

    def coef_c(self, pos):
        self.CheckMeshPos(pos)
        return self.therm_cond_coef(self.MESH_STEP * (pos - 0.5))

    def coef_d(self, pos):
        self.CheckMeshPos(pos)
        return - self.heat_source_coef(pos * self.MESH_STEP) * (self.MESH_STEP ** 2)
    
    
    def TriDiagMatrixAlg(self):

        L = self.NODES_NUM - 1
        
        self.alpha[1] = - self.coef_a(1) / self.coef_b(1)
        self.beta [1] = (self.coef_d(1) - self.coef_c(1) * LEFT_BOUND) / self.coef_b(1)

        self.alpha[L - 1] = - self.coef_c(L - 1) / self.coef_b(L - 1)
        self.beta [L - 1] = (self.coef_d(L - 1) - self.coef_c(L - 1) * RIGHT_BOUND) / self.coef_b(L - 1)

        for i in range(2, self.NODE_BEF_GAP):
            self.alpha[i] = - self.coef_a(i) / (self.coef_b(i) + self.coef_c(i) * self.alpha[i - 1])
            self.beta [i] = (self.coef_d(i) - self.coef_c(i) * self.beta [i - 1]) / \
                            (self.coef_b(i) + self.coef_c(i) * self.alpha[i - 1])

        for i in range(L - 2, self.NODE_AFT_GAP, -1):
            self.alpha[i] = - self.coef_c(i) / (self.coef_b(i) + self.coef_a(i) * self.alpha[i + 1])
            self.beta [i] = (self.coef_d(i) - self.coef_a(i) * self.beta [i + 1]) / \
                            (self.coef_b(i) + self.coef_a(i) * self.alpha[i + 1])
            
        self.sol[0] = LEFT_BOUND
        self.sol[L] = RIGHT_BOUND    

        x_bef_gap = self.NODE_BEF_GAP * self.MESH_STEP
        x_aft_gap = self.NODE_AFT_GAP * self.MESH_STEP
        
        self.sol[self.NODE_BEF_GAP] = (self.therm_cond_coef(x_bef_gap) * self.beta[self.NODE_BEF_GAP - 1] + \
                                       self.therm_cond_coef(x_aft_gap) * self.beta[self.NODE_AFT_GAP + 1]) / \
                                      (self.therm_cond_coef(x_bef_gap) * (1 - self.alpha[self.NODE_BEF_GAP - 1]) + \
                                       self.therm_cond_coef(x_aft_gap) * (1 - self.alpha[self.NODE_AFT_GAP + 1]))

        self.sol[self.NODE_AFT_GAP] = self.sol[self.NODE_BEF_GAP]

        #self.sol[self.NODE_BEF_GAP - 1] = self.alpha[self.NODE_BEF_GAP - 1] * self.sol[self.NODE_BEF_GAP] + \
        #                                  self.beta[self.NODE_BEF_GAP - 1]
        #self.sol[self.NODE_AFT_GAP + 1] = self.alpha[self.NODE_AFT_GAP + 1] * self.sol[self.NODE_AFT_GAP] + \
        #                                  self.beta[self.NODE_AFT_GAP + 1] 

        for i in range(self.NODE_BEF_GAP - 1, 0, -1):
            #print i
            self.sol[i] = self.alpha[i] * self.sol[i + 1] + self.beta[i]

        for i in range(self.NODE_AFT_GAP + 1, L):
            #print i
            self.sol[i] = self.alpha[i] * self.sol[i - 1] + self.beta[i]

        #print self.alpha
        #print self.beta
        #print self.sol
        return

            
    def PrintSol(self, entries_num = 11, to_print_analyt = False, to_print_opt = False):
        if not isinstance(entries_num, int):
            raise ValueError("Incorrect type of entries_num parameter in function PrintSol.") 
        if not isinstance(to_print_analyt, bool):
            raise ValueError("Incorrect type of to_print_analyt parameter in function PrintSol.") 
        if not isinstance(to_print_opt, bool):
            raise ValueError("Incorrect type of to_print_opt parameter in function PrintSol.") 
        if to_print_analyt == True and to_print_opt == True:
            raise ValueError("Cannot print both analytical and optimal numerical solutions in function PrintSol.") 
                    
        dash = '-' * 100
        
        header_str = '{:<5s}   {:<5s}   {:<5s}'   + '   {:<6s}  '
        data_str   = '{:<3d}   {:<5d}   {:>1.3f}' + '   {:>.6f}'
        
        if to_print_analyt == True or to_print_opt:
            header_str += 3 * '   {:<6s}  '
            data_str   += 2 * '   {:>.6f}' + '   {:<.6f}'
        
        sol_an = 0.0
        sol_num = 0.0
        r = 0.0
        d = 0
        
        prob = 0

        if to_print_opt == True:
            prob = StatHeatCondProb(self.NODES_NUM * 2, self.therm_cond_coef, self.q_coef, self.heat_source_coef)
            prob.TriDiagMatrixAlg()
        
        print "MESH_STEP = ", self.MESH_STEP
        print "NODE_BEF_GAP = ", self.NODE_BEF_GAP, self.NODE_BEF_GAP * self.MESH_STEP
        print "NODE_AFT_GAP = ", self.NODE_AFT_GAP, self.NODE_AFT_GAP * self.MESH_STEP
        print "NODES_NUM = ", self.NODES_NUM
        print " "
        
        i = 0.0
        I = 0
        j = 0
        
        print_step = 1.0 
        sec_pr_st  = 1.0
            
       # if self.NODES_NUM > entries_num + 1:
        #    print_step = (self.NODES_NUM - 1.0) / entries_num
         #   sec_pr_st  = (2 * self.NODES_NUM - 1.0) / entries_num
        
        
        while i < self.NODES_NUM:
            if i == 0:
                print dash
                print(header_str.format("№", "n", "x", "y", "y*", "|y-y*|", "d"))
                print dash
                        
            x        = self.MESH_STEP * int(round(i))
            cur_node = int(round(i))
            sol_num  = self.sol[int(round(i))]
            
            if to_print_analyt == True:
                sol_an   = AnalytSol(x)
                    
            if to_print_opt == True:
                sol_an   = prob.sol[int(round(I))]
            
            r = abs(sol_an - sol_num)
            if sol_num != 0:
                d = (r / sol_num)

            print(data_str.format(j, cur_node, x, sol_an, sol_num, r, d))
                  
            i += print_step
            I += sec_pr_st
            j += 1

        x = 0.5
        cur_node = int(round(x / self.MESH_STEP))

        if to_print_analyt == True:
            sol_an   = AnalytSol(x)

        if to_print_opt == True:
            sol_an   = prob.sol[int(round(I))]
        
        #print self.sol
        
        return     

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(print "MESH_STEP = ", self.MESH_STEP)? (<ipython-input-8-dd0705048b33>, line 136)

In [ ]:
mod_prob = StatHeatCondProb(81922, ThermCondCoef_model, QCoef_model, HeatSourceFunc_model)
mod_prob.TriDiagMatrixAlg()

mod_prob.PrintSol(to_print_analyt = True)

In [9]:
# Численное решение задачи и сравнение с численным решением на вдвое более мелкой сетке

N = 1000
i = 0

while i < 5:
    i += 1

    prob = StatHeatCondProb(N, ThermCondCoef, QCoef, HeatSourceFunc)
    prob.TriDiagMatrixAlg()
    prob.PrintSol(to_print_opt = True)

    N *= 2
    
    print '\n\n'

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(int '\n\n')? (<ipython-input-9-adeb057e8922>, line 15)